# Example Go Wrapper for Seldon Core

This notebook goes through the steps to test the Go wrapper. This is presently an example of how to wrap code to run in Go in Seldon. 



Below shows example code for a REST and gRPC server in Go that handles the Seldon Core microservice API for MODELs.

In [1]:
!pygmentize server.go

package main

import (
	"context"
	"flag"
	"fmt"
	"github.com/golang/protobuf/jsonpb"
	"github.com/gorilla/mux"
	"github.com/seldonio/seldon-core/examples/wrappers/go/pkg/api"
	"google.golang.org/grpc"
	"log"
	"net"
	"net/http"
	"os"
	"strconv"
)

var (
	port       = flag.Int("port", 10000, "The server port")
	serverType = flag.String("server_type", "grpc", "The type of server grpc or rest")
)

// One struct for each type of Seldon Server. Here we just create one for MODELs
type ModelServer struct {
}

// Example Predict call with SeldonMessage proto
func  (s *ModelServer) Predict(ctx context.Context, m *api.SeldonMessage) (*api.SeldonMessage, error){

	test := &api.SeldonMessage{
		Status: &api.Status{
			Code:                 0,
			Info:                 "",
			Reason:               "",
			Status:               0,

		},
		Meta: &api.Meta{
			Puid:                 "",
			Tags:                 nil,
			Routing:              nil,
			RequestPath:          nil,
			Metrics:              nil,

## Local Docker Test

For this test we will use the code as is which returns a hard-wired result. For real use you should copy this Go project to build your own components.


In [2]:
!make build_docker

docker build -t seldonio/gomodel:0.1 .
Sending build context to Docker daemon  486.9kB
Step 1/7 : FROM golang
latest: Pulling from library/golang

55d5a1d1: Pulling fs layer 
80d00ae9: Pulling fs layer 
b3117dca: Pulling fs layer 
a19181b2: Pulling fs layer 
759e848f: Pulling fs layer 
9eaeed19: Pulling fs layer 
Digest: sha256:5825f1549d8929cb8bc9525b372668105cf6517df5cd8c7f5b5e3a51673e0e07Downloading  67.18MB/127.9MB
Status: Downloaded newer image for golang:latest
 ---> 7ced090ee82e
Step 2/7 : RUN go get -u google.golang.org/grpc
 ---> Running in 54349cbf622c
Removing intermediate container 54349cbf622c
 ---> 870c6a74226b
Step 3/7 : RUN go get -u github.com/gorilla/mux
 ---> Running in 38b37c710bdf
Removing intermediate container 38b37c710bdf
 ---> e1d97f4a05a0
Step 4/7 : WORKDIR /go/src/github.com/seldonio/seldon-core/examples/wrappers/go
 ---> Running in 177cbd8d0228
Removing intermediate container 177cbd8d0228
 ---> 3cac5de40b35
Step 5/7 : COPY . .
 ---> 042fc193a467
Step 6/7 : R

Run a REST test

In [3]:
!make test_docker_rest

docker run -d --name "gomodel" -p 10000:10000 -e SERVER_TYPE='rest' --rm seldonio/gomodel:0.1
ffe632b3eeaf06c3da6a3bd1ba00a1f036742554705d5b606a5f6338d7b4b818


In [4]:
!seldon-core-tester contract.json 0.0.0.0 10000 -p

----------------------------------------
SENDING NEW REQUEST:

[[4.954]]
RECEIVED RESPONSE:
status {
}
meta {
}
data {
  tensor {
    shape: 1
    shape: 1
    values: 1.0
    values: 3.0
  }
}




In [5]:
!docker rm -f gomodel

gomodel


Run a gRPC test

In [6]:
!make test_docker_grpc

docker run -d --name "gomodel" -p 10000:10000 --rm seldonio/gomodel:0.1
be15ba3b8a729321063bb941be7501c7aec0e7d4f309fc9a95bb56481d18a616


In [7]:
!seldon-core-tester contract.json 0.0.0.0 10000 -p --grpc --tensor

----------------------------------------
SENDING NEW REQUEST:

[[3.577]]
RECEIVED RESPONSE:
status {
}
meta {
}
data {
  tensor {
    shape: 1
    shape: 1
    values: 1.0
    values: 3.0
  }
}




In [8]:
!docker rm -f gomodel

gomodel


## Test in Minikube

In [ ]:
!minikube start --memory 4096

## Setup Seldon Core

Use the setup notebook to [Setup Cluster](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html#Setup-Cluster) with [Ambassador Ingress](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html#Ambassador) and [Install Seldon Core](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html#Install-Seldon-Core). Instructions [also online](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html).

In [16]:
!eval $(minikube docker-env) && make build_docker

docker build -t seldonio/gomodel:0.1 .
Sending build context to Docker daemon  543.7kB
Step 1/7 : FROM golang
latest: Pulling from library/golang

55d5a1d1: Pulling fs layer 
80d00ae9: Pulling fs layer 
b3117dca: Pulling fs layer 
a19181b2: Pulling fs layer 
759e848f: Pulling fs layer 
9eaeed19: Pulling fs layer 
Digest: sha256:5825f1549d8929cb8bc9525b372668105cf6517df5cd8c7f5b5e3a51673e0e07Downloading  35.83MB/45.34MB
Status: Downloaded newer image for golang:latest
 ---> 7ced090ee82e
Step 2/7 : RUN go get -u google.golang.org/grpc
 ---> Running in c47778d3b2ed
Removing intermediate container c47778d3b2ed
 ---> ca1dff8faf05
Step 3/7 : RUN go get -u github.com/gorilla/mux
 ---> Running in e8ab459e22ab
Removing intermediate container e8ab459e22ab
 ---> ff67a9ffe7c8
Step 4/7 : WORKDIR /go/src/github.com/seldonio/seldon-core/examples/wrappers/go
 ---> Running in b5bd80b58c56
Removing intermediate container b5bd80b58c56
 ---> 220974fdf4f9
Step 5/7 : COPY . .
 ---> 6cdc57523d88
Step 6/7 : R

In [17]:
!kubectl create -f resources/deployment_example.json

seldondeployment.machinelearning.seldon.io/example-go created


In [19]:
!kubectl rollout status deploy/go-deployment-go-predictor-27dfae4

Waiting for deployment "go-deployment-go-predictor-27dfae4" rollout to finish: 0 of 1 updated replicas are available...
deployment "go-deployment-go-predictor-27dfae4" successfully rolled out


In [22]:
!seldon-core-api-tester contract.json `minikube ip` `kubectl get svc ambassador -o jsonpath='{.spec.ports[0].nodePort}'` \
    example-go --namespace default -p

----------------------------------------
SENDING NEW REQUEST:

[[7.961]]
RECEIVED RESPONSE:
status {
}
meta {
  puid: "p4vmdh0epbh4o4vtpu469s1ecc"
  requestPath {
    key: "gomodel"
    value: "seldonio/gomodel:0.1"
  }
}
data {
  tensor {
    shape: 1
    shape: 1
    values: 1.0
    values: 3.0
  }
}




In [ ]:
!minikube delete